In [ ]:
%matplotlib widget

import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Noto Sans CJK JP']  # Ubuntu系统

# ------------------------------
# (1) 全局配置：速度区间 & 加减速度
# ------------------------------
# 用列表定义若干区间(单位: km/h)，以及在该区间下的加速度(单位: m/s^2)
# 例如：
#   -   0 ~  50 km/h： 1.0 m/s^2
#   -  50 ~ 160 km/h： 0.8 m/s^2
#   - 160+         ： 0.5 m/s^2

ACC_CONF = [
    (0,   50,   1.0),
    (50,  160,  0.8),
    (160, 500, 0.5)  # 速度上限可设极大
]

# 在离限速 x km/h 范围内时，会将速度直接视为限速，避免小幅反复震荡
SPEED_CLAMP_MARGIN = 0.5  # 0.5 km/h，可根据需求调大或调小

# 若需判断速度转折时，判定加速/减速/匀速的阈值(单位: m/s)
# 用于比较 v2 - v1 的绝对差值是否接近0
SPEED_STATE_TOL = 0.01

# ------------------------------
# 工具函数：速度单位换算
# ------------------------------
def kmph_to_mps(v_kmph: float) -> float:
    """将速度从 km/h 转换为 m/s。"""
    return v_kmph / 3.6

def mps_to_kmph(v_mps: float) -> float:
    """将速度从 m/s 转换为 km/h。"""
    return v_mps * 3.6

# ------------------------------
# 工具函数：根据“当前速度 km/h”找到加速度(绝对值) m/s^2
# ------------------------------
def lookup_acc_abs(v_kmph: float) -> float:
    """
    从 ACC_CONF 里找到当前速度所在区间，然后返回该区间对应的加速度(绝对值)。
    速度区间可能较多，依次扫描即可。
    """
    for (low, high, acc_val) in ACC_CONF:
        if low <= v_kmph < high:
            return acc_val
    # 如果没匹配到，返回一个兜底值 (理论不会走到这)
    return ACC_CONF[-1][2]

# ------------------------------
# 1. 读取Excel数据
# ------------------------------
file_path = r"../docs/跨线运行SPCK曲线定义组.xlsx"
df = pd.read_excel(file_path, sheet_name="自定义线路")

# 只保留 Par1、运行速度kmph 非空的行，避免后续计算出错
df = df.dropna(subset=["Par1", "运行速度kmph"])

# ------------------------------
# 2. 设置仿真初始条件
# ------------------------------
v0 = kmph_to_mps(10.0)  # 初始速度 (10 km/h -> m/s)
dt = 0.1               # 时间步长 (s)

time_list = [0.0]       # 每一步(时间步)的累计时间
speed_list = [v0]       # 每一步(时间步)的列车速度 (m/s)
distance_list = [0.0]   # 每一步(时间步)的累计里程 (m)

# 记录对应时刻的限速(km/h)——仅为绘图或对比需要
initial_limit_kmph = df.iloc[0]["运行速度kmph"]
speed_limit_list = [initial_limit_kmph]

total_time = 0.0
total_distance = 0.0

# 记录当前速度，用于衔接各段
v_current = v0

# ------------------------------
# 3. 按分段进行仿真 (时间步进法 + 限速死区抑制震荡)
# ------------------------------
for idx, row in df.iterrows():
    segment_id = row["线路分段"]
    line_type = row["平面线路类型"]
    L_segment = row["Par1"]           # 本段长度 (m)
    V_lim_kmph = row["运行速度kmph"]  # 本段限速 (km/h)
    V_lim_mps  = kmph_to_mps(V_lim_kmph)

    distance_in_segment = 0.0  # 本段内已行驶距离

    while distance_in_segment < L_segment:
        # (1) 根据当前速度(单位: m/s)得到所在区间加速度(绝对值)
        v_cur_kmph = mps_to_kmph(v_current)
        acc_abs = lookup_acc_abs(v_cur_kmph)

        # (2) 判断加/减/匀速
        speed_diff_from_lim = v_current - V_lim_mps

        # 2.1 若已接近限速 (如在 +/- SPEED_CLAMP_MARGIN km/h 范围内)，则直接设置为匀速
        #     防止出现速度在限速上下抖动
        #     注：speed_diff_from_lim 单位是 m/s，需要先转 km/h 再比
        diff_kmh = abs(mps_to_kmph(v_current) - V_lim_kmph)
        if diff_kmh <= SPEED_CLAMP_MARGIN:
            # 强制贴合限速
            a = 0.0
            v_next = V_lim_mps
        else:
            # 2.2 未在死区内，正常计算
            if abs(speed_diff_from_lim) < 1e-3:
                # 与限速极其接近(小于 ~0.0036km/h)，视为匀速
                a = 0.0
            elif speed_diff_from_lim < 0:
                # v_current < v_lim => 加速
                a = +acc_abs
            else:
                # v_current > v_lim => 减速
                a = -acc_abs

            # 计算下一时刻速度
            v_next = v_current + a * dt
            if v_next < 0:
                v_next = 0.0

        # (3) 预测下一时刻在本段内的位移
        s_next = distance_in_segment + v_next * dt

        # (4) 若下一步会超段尾，则需缩短 dt
        if s_next > L_segment:
            remain = L_segment - distance_in_segment
            v_avg = 0.5*(v_current + v_next)
            if v_avg < 1e-6:
                dt_part = remain / max(v_current, 1e-6)
            else:
                dt_part = remain / v_avg

            v_new = v_current + a * dt_part
            if v_new < 0:
                v_new = 0.0

            total_time += dt_part
            total_distance += remain
            distance_in_segment = L_segment  # 到段尾

            time_list.append(total_time)
            speed_list.append(v_new)
            distance_list.append(total_distance)
            speed_limit_list.append(V_lim_kmph)

            v_current = v_new
        else:
            # (5) 正常一步
            total_time += dt
            total_distance += v_next * dt
            distance_in_segment = s_next

            time_list.append(total_time)
            speed_list.append(v_next)
            distance_list.append(total_distance)
            speed_limit_list.append(V_lim_kmph)

            v_current = v_next

# ------------------------------
# 4. 将结果组装为 DataFrame (完整时间步进)
# ------------------------------
df_result = pd.DataFrame({
    "time_s": time_list,
    "speed_m_s": speed_list,
    "speed_km_h": [mps_to_kmph(v) for v in speed_list],
    "distance_m": distance_list,
    "limit_km_h": speed_limit_list
})

# ==================================================
# 5. 提取 “速度转折点”（加/减/匀速三种状态变化）
# ==================================================

def get_motion_state(v1, v2, tol=0.01):
    """
    根据相邻两个速度 (m/s) 的差值判断处于加速/减速/匀速状态。
      - v2 - v1 > tol:    加速(acc)
      - v2 - v1 < -tol:   减速(dec)
      - 其余:             匀速(const)
    """
    diff = v2 - v1
    if diff > tol:
        return "acc"
    elif diff < -tol:
        return "dec"
    else:
        return "const"

motion_states = []
for i in range(len(df_result) - 1):
    v_cur = df_result.loc[i, "speed_m_s"]
    v_next = df_result.loc[i+1, "speed_m_s"]
    state = get_motion_state(v_cur, v_next, tol=SPEED_STATE_TOL)
    motion_states.append(state)
# 最后一个点无法用 v[i+1] - v[i]，可认为与前一状态相同
motion_states.append(motion_states[-1])
df_result["state"] = motion_states

# 5.1 找出“速度转折点”的索引
turning_indices = [0]
for i in range(1, len(df_result)):
    if df_result.loc[i, "state"] != df_result.loc[i-1, "state"]:
        turning_indices.append(i)
if turning_indices[-1] != len(df_result)-1:
    turning_indices.append(len(df_result)-1)
turning_indices = sorted(set(turning_indices))

# 5.2 去掉与前后点速度几乎相同的“伪拐点”
filtered_indices = []
for idx_i in turning_indices:
    if idx_i == 0 or idx_i == len(df_result)-1:
        filtered_indices.append(idx_i)
    else:
        spd_i   = df_result.loc[idx_i, "speed_km_h"]
        spd_prev= df_result.loc[idx_i-1, "speed_km_h"]
        spd_next= df_result.loc[idx_i+1, "speed_km_h"]
        if (abs(spd_i - spd_prev) < 1e-3) and (abs(spd_i - spd_next) < 1e-3):
            pass
        else:
            filtered_indices.append(idx_i)
filtered_indices = sorted(set(filtered_indices))

df_turning_points = df_result.loc[filtered_indices,
    ["time_s","speed_m_s","speed_km_h","distance_m"]].copy()
df_turning_points.reset_index(drop=True, inplace=True)

print("===== 速度转折点(含段首/段中/段尾)列表 =====")
print(df_turning_points)

# ------------------------------
# 6. 作图：完整曲线 + 转折点
# ------------------------------
plt.figure(figsize=(8, 5))
plt.plot(df_result["time_s"], df_result["speed_km_h"], label="列车实际速度(全)")
plt.plot(df_result["time_s"], df_result["limit_km_h"], label="线路限速", linestyle="--")
plt.scatter(df_turning_points["time_s"],
            df_turning_points["speed_km_h"],
            color='red', marker='o', label="速度转折点")

plt.xlabel("时间 (s)")
plt.ylabel("速度 (km/h)")
plt.title("列车速度-时间曲线 (含转折点)，带限速死区抑制震荡")
plt.grid(True)
plt.legend()
plt.show()
